In [2]:
import satpy
import numpy
import matplotlib.pyplot as plt
import xarray
import pandas
import earthaccess
import os
from pathlib import Path
from datetime import datetime


Plot satellite orbits historically based on TLE and using swath width find intersections of all three satellites within a certain time distinction... convert code to a class